In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
X_full = pd.read_csv("../input/titanic/train.csv")
X_test = pd.read_csv("../input/titanic/test.csv")
ids = X_test['PassengerId']
ids.shape
X_test.shape

(418, 11)

In [4]:
X_full.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
X_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
#Substitua os valores nulos por um valor que não esteja presente naquela coluna dos dados, 
#se o seu modelo for de árvores de decisão. Dessa maneira você está informando o modelo que aquele valor é diferente de todos os outros.
#Filho, Mario. Manual Prático de Data Science (pp. 66-67). Kindle Edition. 

X_test.fillna(-9999, inplace=True)
X_full.fillna(-9999, inplace=True)

In [7]:
# Train Data & Target
X = X_full.drop(['Survived'],axis=1)
y = X_full['Survived']

cat_cols = X_full.select_dtypes(include='object').columns.values
num_cols = X_full.select_dtypes(exclude='object').columns.values
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,-9999,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,-9999,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,-9999,S


In [8]:
#choose the features we want to train, just forget the float data
cate_features_index = np.where(X.dtypes != float)[0]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        891 non-null    object 
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [9]:
X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=42,train_size=.85)
X.columns.values[cate_features_index]

array(['PassengerId', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket',
       'Cabin', 'Embarked'], dtype=object)

In [10]:
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [11]:
model.fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))

Learning rate set to 0.029583
0:	learn: 0.7926024	test: 0.7686567	best: 0.7686567 (0)	total: 64.6ms	remaining: 1m 4s
1:	learn: 0.7978864	test: 0.7611940	best: 0.7686567 (0)	total: 70.4ms	remaining: 35.1s
2:	learn: 0.8044914	test: 0.7835821	best: 0.7835821 (2)	total: 79.7ms	remaining: 26.5s
3:	learn: 0.8097754	test: 0.7910448	best: 0.7910448 (3)	total: 83.7ms	remaining: 20.8s
4:	learn: 0.8044914	test: 0.7835821	best: 0.7910448 (3)	total: 84.8ms	remaining: 16.9s
5:	learn: 0.7965654	test: 0.7835821	best: 0.7910448 (3)	total: 85.8ms	remaining: 14.2s
6:	learn: 0.8044914	test: 0.7835821	best: 0.7910448 (3)	total: 91.6ms	remaining: 13s
7:	learn: 0.8071334	test: 0.7910448	best: 0.7910448 (3)	total: 97.7ms	remaining: 12.1s
8:	learn: 0.8084544	test: 0.7985075	best: 0.7985075 (8)	total: 104ms	remaining: 11.5s
9:	learn: 0.8124174	test: 0.8059701	best: 0.8059701 (9)	total: 108ms	remaining: 10.7s
10:	learn: 0.8177015	test: 0.8059701	best: 0.8059701 (9)	total: 114ms	remaining: 10.3s
11:	learn: 0.8150

In [12]:
test_preds = model.predict(X_test)

In [13]:
submission = pd.DataFrame({
    'PassengerId':ids,
    'Survived': test_preds
})

In [14]:
submission.to_csv('submission.csv', index=False)

In [15]:
submission.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
